# Running CEDS Scenarios

In this notebook we document how to process and run data from the CEDS database.

In [1]:
%matplotlib inline

from os import listdir
from os.path import join, dirname
from pprint import pprint

import pandas as pd
import pyam
from pyam.utils import LONG_IDX
import pint
from pint.pandas_interface import PintArray

import pymagicc
from pymagicc.io import MAGICCData

import matplotlib.pyplot as plt
plt.style.use('bmh') 

import expectexception

<IPython.core.display.Javascript object>

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
TEST_DATA_PATH = join("..", "tests", "test_data")

## Reading in a CEDS csv

To read in CEDS csv's, we make use of the `pyam` library which is specifically designed for this purpose.

In [4]:
def read_ceds_csv(file_to_read):
    return pyam.IamDataFrame(
        data=file_to_read,
        encoding="utf-8"
    )

ceds_pyam_df = read_ceds_csv(join(TEST_DATA_PATH, "ceds-format-example.csv"))
ceds_pyam_df  # this just shows the type of ceds_pyam_df
ceds_pyam_df.data  # this returns the underlying DataFrame

INFO:root:Reading `../tests/test_data/ceds-format-example.csv`


,model,scenario,region,variable,unit,year,value
0,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5ASIA,Emissions|BC,Mt BC/yr,2015,36.398915
91,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5LAM,Emissions|BC,Mt BC/yr,2015,26.494276
174,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5MAF,Emissions|BC,Mt BC/yr,2015,30.456913
257,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5OECD,Emissions|BC,Mt BC/yr,2015,32.513038
348,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5REF,Emissions|BC,Mt BC/yr,2015,18.561100
439,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,World,Emissions|BC,Mt BC/yr,2015,152.232300
2276,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5ASIA,Emissions|BC,Mt BC/yr,2020,26.645995
2367,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5LAM,Emissions|BC,Mt BC/yr,2020,4.195137
2450,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5MAF,Emissions|BC,Mt BC/yr,2020,24.990606
2533,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5OECD,Emissions|BC,Mt BC/yr,2020,20.422119


## Checking an `IamDataFrame`

It is very easy to check that the sum of a given variable's sub-categories is equal to its declared total and that the sum of regions gives the world total.

We show how in the next cell.

In [5]:
# show check_internal_consistency method here

## Reshaping an `IamDataFrame`

Here we show how to reshape an `IamDataFrame` to get it into the format expected by `openscm` so we can then write files with the data in it.

Note: we normally want to take this step last, after we have done all our aggregation etc., as it means that we no longer have an `IamDataFrame` and can't use all the helpful tools it provides any more.

In [6]:
def reshape_pyam_df_to_openscm_df(pyam_df):
    raw_df = pyam_df.data
    
    reindexed_df = raw_df.set_index(
        ["model", "scenario", "region", "variable", "unit", "year"]
    ).unstack().T
    
    years = reindexed_df.index.get_level_values("year")
    if (years % 1 == 0).all() :
        reindexed_df.index = years.astype(int)
    reindexed_df.index.name = "YEAR"
    
    models = reindexed_df.columns.get_level_values("model")
    scenarios = reindexed_df.columns.get_level_values("scenario")
    regions = reindexed_df.columns.get_level_values("region")
    variables = reindexed_df.columns.get_level_values("variable")
    units = reindexed_df.columns.get_level_values("unit")
    todos = ["SET"] * len(units)
    
    reindexed_df.columns = pd.MultiIndex.from_arrays(
        [models, scenarios, variables, todos, units, regions],
        names=("MODEL", "SCENARIO", "VARIABLE", "TODO", "UNITS", "REGION"),
    )
    
    return reindexed_df

In [7]:
reshape_pyam_df_to_openscm_df(ceds_pyam_df)

MODEL    MODEL-NAME-HYPHENS                                          \
SCENARIO SCENARIO-A-B-CDE-2                                           
VARIABLE       Emissions|BC Emissions|BC|Agricultural Waste Burning   
TODO                    SET                                     SET   
UNITS              Mt BC/yr                                Mt BC/yr   
REGION               R5ASIA                                  R5ASIA   
YEAR                                                                  
2015              36.398915                                3.974090   
2020              26.645995                                2.737676   
2030              36.275098                                9.717128   
2040              32.112875                                8.026567   
2050              32.827797                                1.968237   
2060              47.318307                                3.889015   
2070              30.100152                                2.213773   
2080              27.344638                                7.074890   
2090              36.352506                                7.606149   
2100              31.187290                                4.797244   

MODEL                                                            \
SCENARIO                                                          
VARIABLE Emissions|BC|Energy Sector Emissions|BC|Forest Burning   
TODO                            SET                         SET   
UNITS                      Mt BC/yr                    Mt BC/yr   
REGION                       R5ASIA                      R5ASIA   
YEAR                                                              
2015                       1.310619                    2.652261   
2020                       8.896164                    3.540554   
2030                       3.933159                   -1.479242   
2040                       6.339154                    1.199015   
2050                       8.799231                    0.975675   
2060                       1.607069                    4.169055   
2070                       4.971571                    4.518712   
2080                      -1.760939                    8.024338   
2090                      -1.632714                    2.740800   
2100                       2.411164                    1.578012   

MODEL                                                                   \
SCENARIO                                                                 
VARIABLE Emissions|BC|Grassland Burning Emissions|BC|Industrial Sector   
TODO                                SET                            SET   
UNITS                          Mt BC/yr                       Mt BC/yr   
REGION                           R5ASIA                         R5ASIA   
YEAR                                                                     
2015                           6.632639                       6.185669   
2020                          -1.279472                      -1.203520   
2030                           5.246591                       1.119544   
2040                           4.793242                      -0.491045   
2050                           4.584317                       7.386991   
2060                           7.077807                       6.433131   
2070                           7.762036                       1.085432   
2080                           0.722233                       4.163659   
2090                           1.581080                       5.675770   
2100                           4.848120                       1.165334   

MODEL                                                                         \
SCENARIO                                                                       
VARIABLE Emissions|BC|Peat Burning Emissions|BC|Residential Commercial Other   
TODO                           SET                                       SET   
UNITS                     Mt BC/yr                                  Mt BC

## Super brief intro to pyam

The `pyam` library provides some very natural ways of filtering their DataFrames. These are detailed in [their tutorial](https://github.com/IAMconsortium/pyam/blob/master/tutorial/pyam_first_steps.ipynb). Here we use them to help convert IAM data into the emissions variables, regions and units used by openscm and MAGICC.

In [8]:
tdf = ceds_pyam_df.filter(
    level=1,
    model="MODEL-NAME-HYPHENS",
    scenario="SCENARIO-A-B-CDE-2",
    region="World",
)
tdf.variables()
tdf.regions()
tdf.data.head()

0            Emissions|BC
1          Emissions|C2F6
2          Emissions|CCl4
3           Emissions|CF4
4        Emissions|CFC-11
5       Emissions|CFC-113
6       Emissions|CFC-114
7       Emissions|CFC-115
8        Emissions|CFC-12
9           Emissions|CH3
10      Emissions|CH3CCl3
11        Emissions|CH3Cl
12          Emissions|CH4
13           Emissions|CO
14          Emissions|CO2
15    Emissions|HCFC-141b
16    Emissions|HCFC-142b
17      Emissions|HCFC-22
18          Emissions|HFC
19    Emissions|Halon1202
20    Emissions|Halon1211
21    Emissions|Halon1301
22    Emissions|Halon2402
23          Emissions|N2O
24          Emissions|NH3
25          Emissions|NOx
26           Emissions|OC
27          Emissions|SF6
28       Emissions|Sulfur
29          Emissions|VOC
Name: variable, dtype: object

0    World
Name: region, dtype: object

,model,scenario,region,variable,unit,year,value
439,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,World,Emissions|BC,Mt BC/yr,2015,152.232300
2715,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,World,Emissions|BC,Mt BC/yr,2020,127.122154
4991,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,World,Emissions|BC,Mt BC/yr,2030,177.813834
7267,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,World,Emissions|BC,Mt BC/yr,2040,218.627907
9543,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,World,Emissions|BC,Mt BC/yr,2050,219.680082


## Converting CEDS data to openscm data

In [9]:
# ceds openscm mapping
ceds_openscm_var_mapping = {
    "Aircraft": ["Aircraft"],
    "International Shipping": ["International Shipping"],
    "AFOLULUC": ["Agricultural Waste Burning", "Agriculture", 
                 "Forest Burning", "Grassland Burning", 
                 "Peat Burning", "Aggregate - Agriculture and LUC"],
    "Fossil": ["Energy Sector", "Industrial Sector", 
               "Residential Commercial Other", 
               "Solvents Production and Application", 
               "Transportation Sector", "Waste"]
}
# need better name for this
TMP_INDEX = ['model', 'scenario', 'region', 'year', 'unit']

In [43]:
def convert_ceds_to_openscm_variable(ceds_var):
    raw_ceds_var = ceds_var.replace("Emissions|", "")
    
    special_cases = {
        "Sulfur": "SOX",
    }
    
    if raw_ceds_var in special_cases:
        raw_var = special_cases[raw_ceds_var]
    else:
        raw_var = raw_ceds_var.replace("-", "").upper()
        
    return "Emissions|" + raw_var

def convert_ceds_df_to_openscm_df(pyam_df):
    openscm_df = pyam.IamDataFrame(data=pyam_df.data.copy())
    openscm_df.data.region = openscm_df.data.region.str.upper()
    
    output_df = openscm_df.filter(level=1,).data
    output_df.variable = output_df.variable.apply(convert_ceds_to_openscm_variable)
    
    openscm_df = openscm_df.filter(
        level='1-', 
        keep=False
    )
    
    handled_vars = []
    metadata = {}
    for variable in openscm_df.variables():
        base_var = "|".join(variable.split("|")[:2])
        
        for category, suffixes in ceds_openscm_var_mapping.items():
            openscm_var = "{}|{}".format(
                convert_ceds_to_openscm_variable(variable.split("|")[1]),
                category,
            )
            if openscm_var in handled_vars:
                continue
            handled_vars.append(openscm_var)
            
            contrib_vars = ["{}|{}".format(base_var, s) for s in suffixes]
            
            var_cat_df = openscm_df.data[openscm_df.data.variable.isin(contrib_vars)]
            var_cat_df = pd.DataFrame(var_cat_df.groupby(TMP_INDEX).sum()['value'])
            var_cat_df = pd.concat([var_cat_df], keys=[openscm_var], names=['variable'])

            output_df = pd.concat([output_df, var_cat_df.reset_index()], sort=False)

            metadata[openscm_var] = "Sum of {}".format(", ".join(contrib_vars))
    
    return pyam.IamDataFrame(data=output_df), metadata

In [44]:
openscm_df, metadata = convert_ceds_df_to_openscm_df(ceds_pyam_df)

pprint([v for v in openscm_df.variables()])
pprint([r for r in openscm_df.regions()])
reshape_pyam_df_to_openscm_df(openscm_df)

['Emissions|BC',
 'Emissions|BC|AFOLULUC',
 'Emissions|BC|Aircraft',
 'Emissions|BC|Fossil',
 'Emissions|BC|International Shipping',
 'Emissions|C2F6',
 'Emissions|CCL4',
 'Emissions|CF4',
 'Emissions|CFC11',
 'Emissions|CFC113',
 'Emissions|CFC114',
 'Emissions|CFC115',
 'Emissions|CFC12',
 'Emissions|CH3',
 'Emissions|CH3CCL3',
 'Emissions|CH3CL',
 'Emissions|CH4',
 'Emissions|CH4|AFOLULUC',
 'Emissions|CH4|Fossil',
 'Emissions|CH4|International Shipping',
 'Emissions|CO',
 'Emissions|CO2',
 'Emissions|CO2|AFOLULUC',
 'Emissions|CO2|Aircraft',
 'Emissions|CO2|Fossil',
 'Emissions|CO2|International Shipping',
 'Emissions|CO|AFOLULUC',
 'Emissions|CO|Aircraft',
 'Emissions|CO|Fossil',
 'Emissions|CO|International Shipping',
 'Emissions|HALON1202',
 'Emissions|HALON1211',
 'Emissions|HALON1301',
 'Emissions|HALON2402',
 'Emissions|HCFC141B',
 'Emissions|HCFC142B',
 'Emissions|HCFC22',
 'Emissions|HFC',
 'Emissions|N2O',
 'Emissions|NH3',
 'Emissions|NH3|AFOLULUC',
 'Emissions|NH3|Aircra

MODEL    MODEL-NAME-HYPHENS                                            \
SCENARIO SCENARIO-A-B-CDE-2                                             
VARIABLE       Emissions|BC Emissions|BC|AFOLULUC Emissions|BC|Fossil   
TODO                    SET                   SET                 SET   
UNITS              Mt BC/yr              Mt BC/yr            Mt BC/yr   
REGION               R5ASIA                R5ASIA              R5ASIA   
YEAR                                                                    
2015              36.398915             19.897554           16.501361   
2020              26.645995              9.260125           17.385870   
2030              36.275098             22.694679           13.580419   
2040              32.112875             13.432291           18.680585   
2050              32.827797             13.065412           19.762386   
2060              47.318307             23.685064           23.633243   
2070              30.100152             13.515651           16.584501   
2080              27.344638             16.320317           11.024320   
2090              36.352506             19.514986           16.837519   
2100              31.187290             13.959489           17.227801   

MODEL                                                               \
SCENARIO                                                             
VARIABLE Emissions|CH4 Emissions|CH4|AFOLULUC Emissions|CH4|Fossil   
TODO               SET                    SET                  SET   
UNITS        Mt CH4/yr              Mt CH4/yr            Mt CH4/yr   
REGION          R5ASIA                 R5ASIA               R5ASIA   
YEAR                                                                 
2015         33.186689              11.676655            21.510034   
2020         49.676666              34.772684            14.903982   
2030         46.979559              31.844023            15.135537   
2040         44.691875              11.953285            32.738590   
2050         30.754564              18.804239            11.950325   
2060         45.037278              10.391316            34.645962   
2070         40.648090              13.925268            26.722822   
2080         47.319430              31.660236            15.659194   
2090         35.504855              13.973206            21.531649   
2100         38.099801              11.695808            26.403994   

MODEL                                                     \
SCENARIO                                                   
VARIABLE Emissions|CO Emissions|CO2 Emissions|CO2|Fossil   
TODO              SET           SET                  SET   
UNITS        Mt CO/yr     Mt CO2/yr            Mt CO2/yr   
REGION         R5ASIA        R5ASIA               R5ASIA   
YEAR                                                       
2015        27.647071     36.300442            36.300442   
2020        37.980337     18.820329            18.820329   
2030        56.035339     19.323918            19.323918   
2040        28.729780     29.048607            29.048607   
2050        27.664517     22.267156            22.267156   
2060        29.938876     21.045728            21.045728   
2070        26.393159     32.946365            32.946365   
2080        44.438492     22.693831            22.693831   
2090        55.683231     16.846655            16.846655   
2100        38.403976     15.680066            15.680066   

MODEL                                          ...                   \
SCENARIO                                       ...                    
VARIABLE Emissions|CO|AFOLULUC                 ...                    
TODO                       SET                 ...                    
UNITS                 Mt CO/yr                 ...                    
REGION                  R5ASIA                 ...                    
YEAR                                           ...                    
2015                  3.371103                 ..

In [173]:
emissions = pint.Context('emms')

ureg = pint.UnitRegistry()
Q_ = ureg.Quantity

ureg.add_context(emissions)
ureg.enable_contexts('emms')

ureg.define('yr = year')
ureg.define('kt = 1000 * kilogram')

## Option A: define everything in molecular weight
# downside: can convert e.g. C to O...
# ureg.define('molecular_weight = g / mol = mw')
# ureg.define('C = 12 mw')
# ureg.define('O = 16 mw')
# ureg.define('CO2 = 44 mw')


## Option B: simply define a few customs
# this is my preferred as there's not so many
# conversions and we don't want to teach pint
# to read chemical formulas yet and we should
# restrict what can be converted

ureg.define('C = []')
ureg.define('CO2 = 44/12*C')
ureg.define('O = []')
ureg.define('N = []')
ureg.define('NOx = 46/14*N')
ureg.define('S = []')
ureg.define('SO2 = 2*S')
ureg.define('CH3Br = []')

a = Q_(1, 'C')
b = Q_(1, 'CO2')
# b.to('O')
# b.to('g / mol')
# a.to_base_units()

openscm_df.data.unit = openscm_df.data.unit.str.replace("-", "").replace("Mt CO2equiv/yr", "Mt CO2/yr")
for variable in openscm_df.filter(level=1).variables():
    base_var = variable.split("|")[1]
    conversions = {
        'CCL4': 'CCl4',
        'CH3BR': 'CH3Br',
        'CH3CCL3': 'CH3CCl3',
        'CH3CL': 'CH3Cl',
        'HALON1202': 'Halon1202',
        'HALON1211': 'Halon1211',
        'HALON1301': 'Halon1301',
        'HALON2402': 'Halon2402',
        'HCFC141B': 'HCFC141b',
        'HCFC142B': 'HCFC142b',
        'HCFC141B': 'HCFC141b',
        
    }
    exceptions = ["CO2", "NOx"]
    if base_var in exceptions:
        continue
    if base_var in conversions:
        ureg.define("{} = []".format(conversions[base_var]))
    else:
        ureg.define("{} = []".format(base_var))

tpdf = openscm_df.data.set_index(LONG_IDX).unstack(["variable", "unit"])
tpdf = tpdf.pint.quantify(ureg, level=-1)
tpdf.pint.dequantify().head()
tpdf.pint.to_base_units().pint.dequantify().head()

value                                                    \
              Emissions|BC    Emissions|BC|AFOLULUC    Emissions|BC|Aircraft   
  BC * megametric_ton / yr BC * megametric_ton / yr BC * megametric_ton / yr   
0                36.398915                19.897554                      NaN   
1                26.645995                 9.260125                      NaN   
2                36.275098                22.694679                      NaN   
3                32.112875                13.432291                      NaN   
4                32.827797                13.065412                      NaN   

                                                                               \
       Emissions|BC|Fossil Emissions|BC|International Shipping Emissions|C2F6   
  BC * megametric_ton / yr            BC * megametric_ton / yr C2F6 * kt / yr   
0                16.501361                                 NaN            NaN   
1                17.385870                                 NaN            NaN   
2                13.580419                                 NaN            NaN   
3                18.680585                                 NaN            NaN   
4                19.762386                                 NaN            NaN   

                                                                 \
  Emissions|CCL4 Emissions|CF4 Emissions|CFC11 Emissions|CFC113   
  CCl4 * kt / yr CF4 * kt / yr CFC11 * kt / yr CFC113 * kt / yr   
0            NaN           NaN             NaN              NaN   
1            NaN           NaN             NaN              NaN   
2            NaN           NaN             NaN              NaN   
3            NaN           NaN             NaN              NaN   
4            NaN           NaN             NaN              NaN   

                  ...                                             \
                  ...                              Emissions|SOX   
                  ...                  SO2 * megametric_ton / yr   
0                 ...                                  33.666089   
1                 ...                                  41.844910   
2                 ...                                  34.418179   
3                 ...                                  32.773690   
4                 ...                                  49.689276   

                                                       \
     Emissions|SOX|AFOLULUC    Emissions|SOX|Aircraft   
  SO2 * megametric_ton / yr SO2 * megametric_ton / yr   
0                 19.803034                       NaN   
1                 25.261556                       NaN   
2                 10.807168                       NaN   
3                  9.634049                       NaN   
4                 24.932032                       NaN   

                                                                  \
       Emissions|SOX|Fossil Emissions|SOX|International Shipping   
  SO2 * megametric_ton / yr            SO2 * megametric_ton / yr   
0                 13.863056                                  NaN   
1                 16.583354                                  NaN   
2                 23.611011                                  NaN   
3                 23.139641                                  NaN   
4                 24.757244                                  NaN   

                                                       \
              Emissions|VOC    Emissions|VOC|AFOLULUC   
  VOC * megametric_ton / yr VOC * megametric_ton / yr   
0                 46.654517                 13.807409   
1                 43.523032                 19.877070   
2                 27.218649                  7.511660   
3                 34.581438                 10.996224   
4                 28.498318                 14.305844   

                                                       \
     Emissions|VOC|Aircraft      Emissions|VOC|Fossil   
  VOC * megametric_ton / yr VOC * megametric_ton / yr   
0                  

value                                                \
            Emissions|BC  Emissions|BC|AFOLULUC  Emissions|BC|Aircraft   
  BC * kilogram / second BC * kilogram / second BC * kilogram / second   
0            1153.436659             630.528909                    NaN   
1             844.378672             293.441927                    NaN   
2            1149.513054             719.166354                    NaN   
3            1017.617347             425.652699                    NaN   
4            1040.272340             414.026754                    NaN   

                                                              \
     Emissions|BC|Fossil Emissions|BC|International Shipping   
  BC * kilogram / second              BC * kilogram / second   
0             522.907750                                 NaN   
1             550.936745                                 NaN   
2             430.346700                                 NaN   
3             591.964648                                 NaN   
4             626.245586                                 NaN   

                                                                             \
            Emissions|C2F6           Emissions|CCL4           Emissions|CF4   
  C2F6 * kilogram / second CCl4 * kilogram / second CF4 * kilogram / second   
0                      NaN                      NaN                     NaN   
1                      NaN                      NaN                     NaN   
2                      NaN                      NaN                     NaN   
3                      NaN                      NaN                     NaN   
4                      NaN                      NaN                     NaN   

                                                        \
            Emissions|CFC11           Emissions|CFC113   
  CFC11 * kilogram / second CFC113 * kilogram / second   
0                       NaN                        NaN   
1                       NaN                        NaN   
2                       NaN                        NaN   
3                       NaN                        NaN   
4                       NaN                        NaN   

                  ...                                         \
                  ...                          Emissions|SOX   
                  ...                  S * kilogram / second   
0                 ...                            2133.673568   
1                 ...                            2652.026983   
2                 ...                            2181.339169   
3                 ...                            2077.115508   
4                 ...                            3149.183533   

                                                                       \
  Emissions|SOX|AFOLULUC Emissions|SOX|Aircraft  Emissions|SOX|Fossil   
   S * kilogram / second  S * kilogram / second S * kilogram / second   
0            1255.067361                    NaN            878.606208   
1            1601.015006                    NaN           1051.011977   
2             684.931626                    NaN           1496.407543   
3             610.582241                    NaN           1466.533267   
4            1580.130592                    NaN           1569.052941   

                                                                \
  Emissions|SOX|International Shipping           Emissions|VOC   
                 S * kilogram / second VOC * kilogram / second   
0                                  NaN             1478.424016   
1                                  NaN             1379.191130   
2                                  NaN              862.525371   
3                                  NaN             1095.843055   
4                                  NaN              903.076488   

                                                                           \
   Emissions|VOC|AFOLULUC  Emissions|VOC|Aircraft    Emissions|VOC|Fossil   
  VOC * kilogram / second VOC * ki

## Converting openscm data to MAGICC data

Here we show how to then convert an openscm data table to MAGICC data.

In [ ]:
def convert_openscm_to_magicc_variable(ceds_variable):
    category_codes = {
        "Aircraft": "AIR",
        "International Shipping": "SHIP",
        "AFOLULUC": "B",
        "Fossil": "I",
    }
    
    special_cases = {
        "VOC": "NMVOC",
    }
    
    # Improvement: do this with regexp
    species = ceds_variable.split("|")[1]
    if species in special_cases:
        species = special_cases[species]
        
    try:
        category = ceds_variable.split("|")[2]
        category_code = category_codes[category]
    except IndexError:
        category_code = ""
    
    return "{}{}_EMIS".format(species, category_code)
    
def convert_openscm_to_magicc_df(openscm_df):
    magicc_df = pyam.IamDataFrame(data=openscm_df.data.copy())
    magicc_df.data.variable = magicc_df.data.variable.apply(convert_openscm_to_magicc_variable)
    # unit conversions
    return magicc_df

In [ ]:
magicc_df = convert_openscm_to_magicc_df(openscm_df)
reshape_pyam_df_to_openscm_df(magicc_df)

### Converting to SCEN7 format

In [ ]:
def convert_bunkers_to_magicc6_variable(bunker_variable):
    return bunker_variable.replace("SHIP", "I").replace("AIR", "I")

def get_bunkers_df_from_magicc_df(magicc_df):
    ship_df = magicc_df.filter(variable="*SHIP*").data
    ship_df.variable = ship_df.variable.apply(convert_bunkers_to_magicc6_variable)
    ship_df.region = "BUNKERS"
    ship_df.set_index(LONG_IDX, inplace=True)
    
    air_df = magicc_df.filter(variable="*AIR*").data
    air_df.variable = air_df.variable.apply(convert_bunkers_to_magicc6_variable)
    air_df.region = "BUNKERS"
    air_df.set_index(LONG_IDX, inplace=True)

    bunkers_df = ship_df + air_df
    bunkers_df.reset_index(inplace=True)
    
    return bunkers_df

def magicc_df_to_scen7_df(magicc_df):
    scen7_df = magicc_df.filter(
        variable=["*SHIP*", "*AIR*"], 
        keep=False
    )
    # strip out all the variables with breakdown
    # data available
    for variable in scen7_df.variables():
        if variable.endswith(("I_EMIS", "B_EMIS")):
            continue

        if variable.replace("_EMIS", "I_EMIS") in scen7_df.variables().tolist():
            scen7_df = scen7_df.filter(
                variable=variable, 
                keep=False
            )
    
    # to dicuss with Malte, should we do this 
    # given I don't think it matters:
    # - add in N2O breakdown
    # - add in CO2B breakdown
    
    scen7_df = scen7_df.data
    bunkers_df = get_bunkers_df_from_magicc_df(magicc_df)
    
    scen7_df = pd.concat([scen7_df, bunkers_df])
    
    return pyam.IamDataFrame(data=scen7_df)

In [ ]:
scen7_df = reshape_pyam_df_to_openscm_df(
    magicc_df_to_scen7_df(magicc_df)
)

#### Write SCEN7 files

In [ ]:
scen7_df

In [ ]:
for label, df in scen7_df.groupby(level=["MODEL", "SCENARIO"], axis=1):
    fn = "{}_{}.SCEN7".format(*label)
    df.columns = df.columns.droplevel("MODEL").droplevel("SCENARIO")
    df.index = df.index.astype(int)
    writer = MAGICCData()
    writer.df = df
    writer.metadata = {
        "header": "required for some reason\n\n"
    }
    writer.write(fn)

### Converting to SCEN format

In [ ]:
def magicc_df_to_scen_df(magicc_df, world_only=True):
    if not world_only:
        raise NotImplementedError("Neccesary checks not yet included")
        
    scen_emis = [
        v + "_EMIS" 
        for v in pymagicc.definitions.scen_emms_code_1]
    scen_df = magicc_df.filter(
        variable=["*SHIP*", "*AIR*"],
        keep=False
    )
    
    scen_df = scen_df.data
    bunkers_df = get_bunkers_df_from_magicc_df(magicc_df)
    
    scen_df = pd.concat([scen_df, bunkers_df])
    
    scen_df = pyam.IamDataFrame(data=scen_df)
    if world_only:
        scen_df = scen_df.filter(
            region="WORLD"
        )
    
    return scen_df.filter(variable=scen_emis)

In [ ]:
scen_iam_df = magicc_df_to_scen_df(magicc_df)
scen_df = reshape_pyam_df_to_openscm_df(scen_iam_df)
scen_iam_df.variables()

In [ ]:
for label, df in scen_df.groupby(level=["MODEL", "SCENARIO"], axis=1):
    fn = "{}_{}.SCEN".format(*label)

    df.columns = df.columns.droplevel("MODEL").droplevel("SCENARIO")
    df.index = df.index.astype(int)
    
    fn
    df
    
    writer = MAGICCData()
    writer.df = df
    writer.metadata = {
        "header": "required for some reason\n\n"
    }
    writer.write(fn)